In [ ]:
import string

# Color levels based on the chart
def get_strength_color(seconds):
    if seconds < 60:  # less than 1 minute
        return "🟣 Very Weak"
    elif seconds < 3600 * 24 * 365:  # less than 1 year
        return "🔴 Weak"
    elif seconds < 3600 * 24 * 365 * 1000:  # less than 1000 years
        return "🟠 Moderate"
    elif seconds < 3600 * 24 * 365 * 100000:  # less than 100k years
        return "🟡 Strong"
    else:
        return "🟢 Very Strong"

# Determine charset size based on password characters
def get_charset_size(password):
    has_lower = any(c in string.ascii_lowercase for c in password)
    has_upper = any(c in string.ascii_uppercase for c in password)
    has_digits = any(c in string.digits for c in password)
    has_symbols = any(c in string.punctuation for c in password)

    size = 0
    if has_lower:
        size += 26
    if has_upper:
        size += 26
    if has_digits:
        size += 10
    if has_symbols:
        size += 33  # Approximate number of printable ASCII symbols

    return size

def password_strength(password):
    attempts_per_second = 1e10  # 10 billion guesses/sec
    length = len(password)
    charset_size = get_charset_size(password)

    if charset_size == 0:
        return "Invalid password"

    total_combinations = charset_size ** length
    crack_time = total_combinations / attempts_per_second

    return get_strength_color(crack_time)

# Example usage
passwords = ["1234", "password", "Pass123", "P@ssw0rd!", "S9&k@t2Y!gBv"]
for pwd in passwords:
    print(f"'{pwd}': {password_strength(pwd)}")

'1234': 🟣 Very Weak
'password': 🟣 Very Weak
'Pass123': 🔴 Weak
'P@ssw0rd!': 🟠 Moderate
'S9&k@t2Y!gBv': 🟢 Very Strong


In [ ]:
!pip install pycryptodome


In [ ]:
import re
import inspect
import base64
import hashlib
from Crypto import Random
from Crypto.Cipher import AES

class PasswordManager():
  def __init__(self, file_name, key):
        self.bs = AES.block_size  # Обычно 16 байт
        self.key = hashlib.sha256(key.encode()).digest()
        self.site = ""
        self.password = ""
        self.file_name = file_name
        with open(f"{self.file_name}.txt", "w") as f:
            f.write("This is your password manager\n")

  def encrypt(self, raw):
      raw = self._pad(raw.encode())  # 🔁 Паддинг сразу на байты
      iv = Random.new().read(AES.block_size)
      cipher = AES.new(self.key, AES.MODE_CBC, iv)
      return base64.b64encode(iv + cipher.encrypt(raw))

  def decrypt(self, enc):
      enc = base64.b64decode(enc)
      iv = enc[:AES.block_size]
      cipher = AES.new(self.key, AES.MODE_CBC, iv)
      decrypted = cipher.decrypt(enc[AES.block_size:])
      return self._unpad(decrypted).decode('utf-8')

  def _pad(self, b):
      pad_len = self.bs - len(b) % self.bs
      return b + bytes([pad_len] * pad_len)

  @staticmethod
  def _unpad(b):
        return b[:-b[-1]]

  def get_password(self):
    caller = inspect.stack()[1].function
    if not caller in dir(PasswordManager):
        self.site=input('Input the website: ')
    with open(f"{self.file_name}.txt", "r") as f:
        for self.line in f:
            if self.line.startswith(f"{self.site}:"):
              self.password=self.line.strip().split(": ")[1]
              break
    if self.password=="" and not caller in dir(PasswordManager):
      print("Password not found")
      return None
    elif self.password=="":
      return None
    else:
      self.password=self.decrypt(self.password)
      return self.password

  @staticmethod
  def generate_passwaord(l_of_p=10, upper_case=True ,lower_case=True,numbers=True,symbols=True):
    passw=[]
    up_case=""
    low_case=""
    num=""
    sym=""
    if upper_case:
      up_case=string.ascii_uppercase[:]
      passw.append(random.choice(up_case))
    if lower_case:
      low_case=string.ascii_lowercase[:]
      passw.append(random.choice(low_case))
    if numbers:
      num=string.digits[:]
      passw.append(random.choice(num))
    if symbols:
      sym=string.punctuation[:]
      passw.append(random.choice(sym))
    all=up_case+low_case+num+sym
    for i in range(l_of_p-upper_case-lower_case-numbers-symbols):
      passw.append(random.choice(all))
    random.shuffle(passw)
    result = "".join(passw)
    print(f"Your new password is: {result}")
    return result

  @staticmethod
  def str_into_bool(string):
    if string=="y":
      return True
    else:
      return False
  def add_change_password(self):
    self.site=input('Input the website: ')
    self.password=""
    if not self.get_password():
      self.password=input('Input the password or generate it by writing generate: ')
      if self.password=="generate":
        self.l_of_p=int(input('Input the length of the password: '))
        self.upper_case= self.str_into_bool(input('Include uppercase letters? (y/n)'))
        self.lower_case=self.str_into_bool(input('Include lowercase letters? (y/n)'))
        self.numbers=self.str_into_bool(input('Include numbers? (y/n)'))
        self.symbols=self.str_into_bool(input('Include symbols? (y/n)'))
        self.password=self.generate_passwaord(self.l_of_p,self.upper_case,self.lower_case,self.numbers,self.symbols)
      self.password = self.encrypt(self.password).decode('utf-8')
      with open(f"{self.file_name}.txt", "a") as f:
        f.write(f"{self.site}: {self.password} \n")
    else:
      conf=input('Do you want to change the password? (y/n)')
      if conf=="y":
        self.password=input('Input the password or generate it by writing generate: ')
        if self.password=="generate":
          self.l_of_p=int(input('Input the length of the password: '))
          self.upper_case= self.str_into_bool(input('Include uppercase letters? (y/n)'))
          self.lower_case=self.str_into_bool(input('Include lowercase letters? (y/n)'))
          self.numbers=self.str_into_bool(input('Include numbers? (y/n)'))
          self.symbols=self.str_into_bool(input('Include symbols? (y/n)'))
          self.password=self.generate_passwaord(self.l_of_p,self.upper_case,self.lower_case,self.numbers,self.symbols)
          self.password = self.encrypt(self.password).decode('utf-8')
          self.updated_lines=[]
        with open(f"{self.file_name}.txt", "r") as file:
          for self.line in file:
            if self.line.startswith(f"{self.site}:"):
              self.updated_lines.append(f"{self.site}: {self.password}\n")
            else:
              self.updated_lines.append(self.line)

        with open(f"{self.file_name}.txt", "w") as file:
          file.writelines(self.updated_lines)

passw=PasswordManager("passwords","uraa")
passw.add_change_password()


Input the website: a
Input the password or generate it by writing generate: generate
Input the length of the password: 4
Include uppercase letters? (y/n)y
Include lowercase letters? (y/n)y
Include numbers? (y/n)y
Include symbols? (y/n)n
Your new password is: fQf4


In [ ]:
print(passw.get_password())

Input the website: a
MLSOj4p"3Y


In [ ]:
passw.add_change_password()

Input the website: b
Input the password: f
